In [2]:
import pandas as pd
from datetime import date, time
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [ ]:
# Ruta al archivo Parquet
ruta_archivo = "..\\datasets\\raw\\taxi_zone_lookup.csv"
# Leer el archivo Parquet en un DataFrame de Polars
taxi_zone = pd.read_csv(ruta_archivo)
# Suponiendo que df es tu DataFrame
taxi_zone.drop(columns=['service_zone'], inplace=True)

In [ ]:
print("Modelo de datos de: tax_izone_lookup.csv ")
print("Cantidad de registros :", taxi_zone.shape[0], "Cantidad de las Columnas :", taxi_zone.shape[1])
print("\n")
print("Schema")
print(taxi_zone.dtypes)
print("\n")
taxi_zone.head(5)

In [24]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
from io import BytesIO

def cargar_parquet_a_blob(df, ruta_destino_blob, account_name, account_key, container_name):
    """
    Función para cargar un DataFrame en formato Parquet a un contenedor de Azure Blob Storage.

    Args:
    df (pandas.DataFrame): El DataFrame que se desea cargar.
    ruta_destino_blob (str): La ruta de destino en el contenedor Blob.
    account_name (str): El nombre de la cuenta de almacenamiento.
    account_key (str): La clave de la cuenta de almacenamiento.
    container_name (str): El nombre del contenedor Blob.

    Returns:
    None
    """
    # Convertir DataFrame a bytes
    bytes_io = BytesIO()
    df.to_parquet(bytes_io)

    # Conexión a la cuenta de almacenamiento
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

    # Subir archivo al contenedor
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(ruta_destino_blob)

    with bytes_io as data:
        blob_client.upload_blob(data.getvalue(), overwrite=True)



In [19]:
# ruta al archivo zip
ruta_zip = '..\\datasets\\raw\\archive.zip'

#lista para almacenar los datos de los archivos csv
datos_csv = []

#abrir el archivo zip y leer los csv
with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    #recorrer cada archivo en el zip
    for nombre_archivo in zip_ref.namelist():
        #extraer archivo
        with zip_ref.open(nombre_archivo) as archivo_csv:
            #leer csv y almacenar datos en un df
            datos = pd.read_csv(archivo_csv, sep=',', encoding='utf-8')
            #agregar los datos a la lista
            datos_csv.append(datos)

#concatenar todos los df en uno solo
df_emisiones = pd.concat(datos_csv, ignore_index = True)


#filtramos por país 'United States' ya que solo nos sirven los registros pertenecientes a este pais
df_emisiones_usa = df_emisiones[df_emisiones['Country'] == 'United States']

#nos sirve en este caso las energías derivadas del petróleo además de las renovables
emisiones_usa = df_emisiones_usa.loc[(df_emisiones_usa['Energy_type'] == 'petroleum_n_other_liquids') | (df_emisiones_usa['Energy_type'] == 'renewables_n_other')]
columns_to_drop = ['Unnamed: 0', 'Country', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP']
emisiones_usa.drop(columns=columns_to_drop, inplace=True)
new_column_names = {
    'Energy_type': 'TipoEnergia',
    'Year': 'Anio',
    'Energy_consumption': 'ConsumoEnergia',
    'Energy_production': 'ProducionEnergia',
    'GDP': 'ConsumoPerCapita',
    'Population': 'Poblacion',
    'CO2_emission': 'CantidadEmisionesCO2'
}

emisiones_usa.rename(columns=new_column_names, inplace=True)
# Lista con el orden deseado de las columnas
new_column_order = ['Anio', 'TipoEnergia', 'ProducionEnergia', 'ConsumoEnergia', 
                    'Poblacion', 'ConsumoPerCapita', 'CantidadEmisionesCO2']

# Reordenar las columnas del DataFrame
emisiones_usa = emisiones_usa.reindex(columns=new_column_order)

In [20]:
#filtramos por país 'United States' ya que solo nos sirven los registros pertenecientes a este pais
df_emisiones_usa = df_emisiones[df_emisiones['Country'] == 'United States']

#nos sirve en este caso las energías derivadas del petróleo además de las renovables
emisiones_usa = df_emisiones_usa.loc[(df_emisiones_usa['Energy_type'] == 'petroleum_n_other_liquids') | (df_emisiones_usa['Energy_type'] == 'renewables_n_other')]
columns_to_drop = ['Unnamed: 0', 'Country', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP']
emisiones_usa.drop(columns=columns_to_drop, inplace=True)
new_column_names = {
    'Energy_type': 'TipoEnergia',
    'Year': 'Anio',
    'Energy_consumption': 'ConsumoEnergia',
    'Energy_production': 'ProducionEnergia',
    'GDP': 'ConsumoPerCapita',
    'Population': 'Poblacion',
    'CO2_emission': 'CantidadEmisionesCO2'
}

emisiones_usa.rename(columns=new_column_names, inplace=True)
# Lista con el orden deseado de las columnas
new_column_order = ['Anio', 'TipoEnergia', 'ProducionEnergia', 'ConsumoEnergia', 
                    'Poblacion', 'ConsumoPerCapita', 'CantidadEmisionesCO2']

# Reordenar las columnas del DataFrame
emisiones_usa = emisiones_usa.reindex(columns=new_column_order)

C:\Users\ozi\AppData\Local\Temp\ipykernel_3160\2888679702.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emisiones_usa.drop(columns=columns_to_drop, inplace=True)
C:\Users\ozi\AppData\Local\Temp\ipykernel_3160\2888679702.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emisiones_usa.rename(columns=new_column_names, inplace=True)


In [21]:
print("Modelo de datos de: tax_izone_lookup.csv ")
print("Cantidad de registros :", emisiones_usa.shape[0], "Cantidad de las Columnas :", emisiones_usa.shape[1])
print("\n")
print("Schema")
print(emisiones_usa.dtypes)
print("\n")
emisiones_usa.head(5)

Modelo de datos de: tax_izone_lookup.csv 
Cantidad de registros : 80 Cantidad de las Columnas : 7


Schema
Anio                      int64
TipoEnergia              object
ProducionEnergia        float64
ConsumoEnergia          float64
Poblacion               float64
ConsumoPerCapita        float64
CantidadEmisionesCO2    float64
dtype: object




,Anio,TipoEnergia,ProducionEnergia,ConsumoEnergia,Poblacion,ConsumoPerCapita,CantidadEmisionesCO2
1323,1980,petroleum_n_other_liquids,20.473758,34.158965,227119.0,7080.75,2455.243565
1325,1980,renewables_n_other,5.428342,5.428342,227119.0,7080.75,0.000000
2709,1981,petroleum_n_other_liquids,20.412303,31.890666,229416.2,7260.45,2244.422124
2711,1981,renewables_n_other,5.413688,5.413688,229416.2,7260.45,0.000000
4095,1982,petroleum_n_other_liquids,20.458149,30.198364,231608.5,7129.56,2101.609960
